In this Notebook are the following things:

1. The Google Maps Distance Matrix for all the citibike stations (walking and bicycling)
https://developers.google.com/maps/documentation/distance-matrix/overview

2. Arrays Indicating the Pick-Up and Drop-Off Points for each station
https://layer.bicyclesharing.net/map/v1/nyc/stations?fbclid=IwAR1Psv8qmg8zSBl1BPLsYRceAGnyaJF_lFv80LTUO78j9ezSWZckArXyyF0

3. Method to restrict the size of the matrix and points to just lower Manhattan

In [88]:
import pandas as pd
import geopy.distance
from itertools import product
import numpy as np
import json

In [89]:
df = pd.read_json('stations.json')

In [90]:
data = df['features']

In [91]:
new = pd.DataFrame.from_dict(data)

In [92]:
clean = pd.DataFrame(columns = ['long','lat','id','action','points'])
for i in range(1588):
    if 'bike_angels_action' in pd.DataFrame(new['features'].iloc[i])['properties'].index:
        long = pd.DataFrame(new['features'].iloc[i])['geometry']['coordinates'][0]
        lat = pd.DataFrame(new['features'].iloc[i])['geometry']['coordinates'][1]
        station_id = pd.DataFrame(new['features'].iloc[i])['properties']['station_id']
        action = pd.DataFrame(new['features'].iloc[i])['properties']['bike_angels_action']
        points = pd.DataFrame(new['features'].iloc[i])['properties']['bike_angels_points']
        clean = clean.append({'long': long,'lat':lat,'id':station_id,'action':action,'points':points},ignore_index=True)    

In [93]:
sort = clean.sort_values(by=['id'])
sort.to_json('sort.json')

In [94]:
reduce = clean[clean['lat'] < 40.766411]
reduce = reduce[reduce['lat'] > 40.701249]
reduce = reduce[reduce['long'] < -73.967222]

In [96]:
reduce.to_json('reduce.json')

In [97]:
long = str(reduce.iloc[0]['long'])
lat = str(reduce.iloc[0]['lat'])
long2 = str(reduce.iloc[4]['long'])
lat2 = str(reduce.iloc[4]['lat'])

In [98]:
long_arr = reduce['long'].to_numpy()
lat_arr = reduce['lat'].to_numpy()
stations = reduce['id'].to_numpy()

In [99]:
distances = np.ones([len(lat_arr),len(lat_arr)])*10000

In [100]:
num = 0
total = 0
for i,j in list(product(range(len(lat_arr)), range(len(lat_arr)))):
    if geopy.distance.distance((lat_arr[i],long_arr[i]),(lat_arr[j],long_arr[j])).km < 1:
        distances[i,j] = 0
        num += 1
    total += 1

In [101]:
np.savetxt('distancematrix.txt',distances)

In [102]:
distances2 = np.insert(distances,0, values=stations , axis=1)
distances2 = np.insert(distances2,0, values=np.append(0,stations), axis=0)
np.savetxt('distance2matrix.txt',distances2)

In [151]:
for i in np.arange(10):
    lat = lat_arr[i]
    long = long_arr[i]
    long_ind = np.where(distances[i,:] == 0)[0]
    
    if len(long_ind) < 25:
        deststr = ""
        for j in list(long_ind):
            long2 = str(long_arr[j])
            lat2 = str(lat_arr[j])
            deststr += str(lat2 + "%2C" + long2 + "%7C")

        url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=" + str(lat) +"%2C" + str(long) + "&destinations=" + deststr +"&mode=bicycling&key="

        payload={}
        headers = {}

        #response = requests.request("GET", url, headers=headers, data=payload)

        with open((str(lat) + '.txt'), 'w') as outfile:
            json.dump(response.text, outfile)

        for j in range(len(list(long_ind))):
            start = i
            end = list(long_ind)[j]
            time = dict(response.json()['rows'][0])['elements'][j]['duration']['value']
            distances[i,end] = time
            
    elif len(long_ind) < 50:
        
        first = long_ind[:25]
        second = long_ind[25:]
        index = [first,second]
        
        for k in range(2):
            ind = index[k]
        
            deststr = ""
            for j in list(ind):
                long2 = str(long_arr[j])
                lat2 = str(lat_arr[j])
                deststr += str(lat2 + "%2C" + long2 + "%7C")

            url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=" + str(lat) +"%2C" + str(long) + "&destinations=" + deststr +"&mode=bicycling&key="

            payload={}
            headers = {}

            #response = requests.request("GET", url, headers=headers, data=payload)

            with open((str(lat) + "-" + str(k) + '.txt'), 'w') as outfile:
                json.dump(response.text, outfile)

            for j in range(len(list(ind))):
                start = i
                end = list(ind)[j]
                time = dict(response.json()['rows'][0])['elements'][j]['duration']['value']
                distances[i,end] = time
                
    else:
        first = long_ind[:25]
        second = long_ind[25:50]
        third = long_ind[50:]
        index = [first,second,third]
        
        for k in range(3):
            ind = index[k]
            
            deststr = ""
            for j in list(ind):
                long2 = str(long_arr[j])
                lat2 = str(lat_arr[j])
                deststr += str(lat2 + "%2C" + long2 + "%7C")

            url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=" + str(lat) +"%2C" + str(long) + "&destinations=" + deststr +"&mode=bicycling&key="

            payload={}
            headers = {}

            #response = requests.request("GET", url, headers=headers, data=payload)

            with open((str(lat) + "-" +  str(k) + '.txt'), 'w') as outfile:
                json.dump(response.text, outfile)

            for j in range(len(list(ind))):
                start = i
                end = list(ind)[j]
                time = dict(response.json()['rows'][0])['elements'][j]['duration']['value']
                distances[i,end] = time
        
    

In [ ]:
#np.savetxt('bicyclingmatrix.csv',distances)
#np.savetxt('bicyclingmatrix.txt',distances)
#distances2 = np.insert(distances,0, values=stations , axis=1)
#distances2 = np.insert(distances2,0, values=np.append(0,stations), axis=0)
#np.savetxt('bicycling2matrix.csv',distances2)
#np.savetxt('bicycling2matrix.txt',distances2)

In [156]:
for i in np.arange(200,373):
    lat = lat_arr[i]
    long = long_arr[i]
    long_ind = np.where(distances[i,:] == 0)[0]
    
    if len(long_ind) < 25:
        deststr = ""
        for j in list(long_ind):
            long2 = str(long_arr[j])
            lat2 = str(lat_arr[j])
            deststr += str(lat2 + "%2C" + long2 + "%7C")

        url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=" + str(lat) +"%2C" + str(long) + "&destinations=" + deststr +"&mode=walking&key="

        payload={}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)

        with open((str(lat)+ 'walk' + '.txt'), 'w') as outfile:
            json.dump(response.text, outfile)

        for j in range(len(list(long_ind))):
            start = i
            end = list(long_ind)[j]
            time = dict(response.json()['rows'][0])['elements'][j]['duration']['value']
            distances[i,end] = time
            
    elif len(long_ind) < 50:
        
        first = long_ind[:25]
        second = long_ind[25:]
        index = [first,second]
        
        for k in range(2):
            ind = index[k]
        
            deststr = ""
            for j in list(ind):
                long2 = str(long_arr[j])
                lat2 = str(lat_arr[j])
                deststr += str(lat2 + "%2C" + long2 + "%7C")

            url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=" + str(lat) +"%2C" + str(long) + "&destinations=" + deststr +"&mode=walking&key="

            payload={}
            headers = {}

            response = requests.request("GET", url, headers=headers, data=payload)

            with open((str(lat) + "-" + 'walk' + str(k) + '.txt'), 'w') as outfile:
                json.dump(response.text, outfile)

            for j in range(len(list(ind))):
                start = i
                end = list(ind)[j]
                time = dict(response.json()['rows'][0])['elements'][j]['duration']['value']
                distances[i,end] = time
                
    else:
        first = long_ind[:25]
        second = long_ind[25:50]
        third = long_ind[50:]
        index = [first,second,third]
        
        for k in range(3):
            ind = index[k]
            
            deststr = ""
            for j in list(ind):
                long2 = str(long_arr[j])
                lat2 = str(lat_arr[j])
                deststr += str(lat2 + "%2C" + long2 + "%7C")

            url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=" + str(lat) +"%2C" + str(long) + "&destinations=" + deststr +"&mode=walking&key="

            payload={}
            headers = {}

            response = requests.request("GET", url, headers=headers, data=payload)

            with open((str(lat) + "-" + 'walk' + str(k) + '.txt'), 'w') as outfile:
                json.dump(response.text, outfile)

            for j in range(len(list(ind))):
                start = i
                end = list(ind)[j]
                time = dict(response.json()['rows'][0])['elements'][j]['duration']['value']
                distances[i,end] = time
        
    

In [157]:
#np.savetxt('walkingmatrix.csv',distances)
#np.savetxt('walkingmatrix.txt',distances)
#distances2 = np.insert(distances,0, values=stations , axis=1)
#distances2 = np.insert(distances2,0, values=np.append(0,stations), axis=0)
#np.savetxt('walking2matrix.csv',distances2)
#np.savetxt('walking2matrix.txt',distances2)

In [158]:
found = distances[(distances[:,:] < 10000) & (distances[:,:] > 0)]
np.size(found)

10831